In [ ]:
from google.colab import drive
import os, ee

drive.mount('/content/drive', force_remount=True)
export_folder = '/content/drive/My Drive/GEE_Exports/'

# Authentication
cloud_project = 'ee-1wangjas2'
try:
    ee.Initialize(project=cloud_project)
    print('Earth Engine initialized successfully.')
except:
    ee.Authenticate()
    ee.Initialize(project=cloud_project)
    print('Earth Engine initialized after authentication.')

# Parameters
country = "Canada"
province = "Alberta"
year = "2022"
months_to_run = {
    "10": (f'{year}-10-01', f'{year}-10-31'),
    "11": (f'{year}-11-01', f'{year}-11-30'),
}
epsm = 1000  # final export resolution

# Load boundaries for Alberta
gaul = ee.FeatureCollection('FAO/GAUL/2015/level2')
canada = gaul.filter(ee.Filter.eq('ADM0_NAME', country))
province_cities = canada.filter(ee.Filter.eq('ADM1_NAME', province))

def estimate_computation_complexity(geometry, scale):
    area = geometry.area(1).getInfo()
    return area / (scale * scale)

def get_optimal_tile_scale(grid_count):
    if grid_count > 1000000:
        return 2
    elif grid_count > 100000:
        return 4
    elif grid_count > 10000:
        return 8
    else:
        return 16

def zonal_stats_with_date(feature_collection, image, tile_scale):
    stats = image.reduceRegions(
        collection=feature_collection,
        reducer=ee.Reducer.mean(),
        scale=epsm,
        tileScale=tile_scale
    )
    return stats.map(lambda f: f.set({
        'longitude': f.geometry().centroid(1).coordinates().get(0),
        'latitude':  f.geometry().centroid(1).coordinates().get(1),
        'date':      ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    }))

def get_nearest_image(col, target_date, window):
    start = target_date.advance(-window, 'day')
    end = target_date.advance(window, 'day')
    filt = col.filterDate(start, end)
    def add_diff(img):
        diff = ee.Number(img.get('system:time_start')).subtract(target_date.millis()).abs()
        return img.set('time_diff', diff)
    nearest = ee.Image(filt.map(add_diff).sort('time_diff').first())
    return nearest

def process_city(feature, start_date, end_date):
    city_name = feature.get('ADM2_NAME').getInfo().replace(" ", "_")
    geom = feature.geometry()
    grid_count = estimate_computation_complexity(geom, epsm)
    tile_scale = get_optimal_tile_scale(grid_count)
    proj = ee.Projection('EPSG:4326').atScale(epsm)
    grids = geom.coveringGrid(proj)

    elevation = ee.Image("USGS/SRTMGL1_003").select(['elevation'])

    def get_composite_for_day(day):
        viirs = get_nearest_image(ee.ImageCollection("NASA/VIIRS/002/VNP13A1"), day, 15) \
                   .select(['NDVI'])
        elev_rs = elevation.resample('bilinear').reproject(proj, None, epsm)
        viirs = viirs.resample('bilinear').reproject(proj, None, epsm)
        comp = ee.Image.cat([elev_rs, viirs]) \
                 .set('system:time_start', day.millis()) \
                 .set('date', day.format('YYYY-MM-dd'))
        return comp

    start = ee.Date(start_date)
    end = ee.Date(end_date).advance(1, 'day')
    num_days = end.difference(start, 'day')
    days = ee.List.sequence(0, num_days.subtract(1)).map(lambda d: start.advance(d, 'day'))
    comps = ee.ImageCollection(days.map(lambda d: get_composite_for_day(ee.Date(d))))
    stats = comps.map(lambda img: zonal_stats_with_date(grids, img, tile_scale)).flatten()

    task = ee.batch.Export.table.toDrive(
        collection=stats,
        description=f'{city_name}_Combined_{start_date}',
        folder=f'CombinedData/{start_date[:7]}',
        fileNamePrefix=f'{city_name}_Combined_{start_date}',
        fileFormat='CSV',
        selectors=['date', 'elevation', 'NDVI', 'longitude', 'latitude']
    )
    task.start()
    print(f"Export started for {city_name} with tileScale {tile_scale}")

# Process all divisions in Alberta
for city in province_cities.toList(province_cities.size()).getInfo():
    city_feature = ee.Feature(city)
    for month, (start_date, end_date) in months_to_run.items():
        province_folder = os.path.join(export_folder, province)
        if not os.path.exists(province_folder): os.makedirs(province_folder)
        year_folder = os.path.join(province_folder, year)
        if not os.path.exists(year_folder): os.makedirs(year_folder)
        month_folder = os.path.join(year_folder, month)
        if not os.path.exists(month_folder): os.makedirs(month_folder)
        dataset_folder = os.path.join(month_folder, 'CombinedData')
        if not os.path.exists(dataset_folder): os.makedirs(dataset_folder)
        os.chdir(dataset_folder)
        print(f'Processing in {os.getcwd()} for {city_feature.get("ADM2_NAME").getInfo()} in month {month}')
        process_city(city_feature, start_date, end_date)

print("Export tasks initiated. Check Earth Engine Tasks tab.")


Mounted at /content/drive
Earth Engine initialized after authentication.
Processing in /content/drive/My Drive/GEE_Exports/Alberta/2022/10/CombinedData for Division No.  1 in month 10
Export started for Division_No.__1 with tileScale 8
Processing in /content/drive/My Drive/GEE_Exports/Alberta/2022/11/CombinedData for Division No.  1 in month 11


KeyboardInterrupt: 